In [98]:
import urllib
import sqlalchemy
import numpy as np
import pandas as pd
from datetime import datetime

import os
import pandas as pd
import numpy as np


from helpers import rutas, validarProduccion
rutaRelativa        = rutas.rutas()
print(rutaRelativa)
estadoProduccion    = validarProduccion.validarProduccion()


from classes import dbAzure, email
dbAzure = dbAzure.DBAzure(estadoProduccion)
Email   = email.Email

print(estadoProduccion)

/Users/camilo/Desktop/PROYECTOS
CONEXION CON EL SERVIDOR
True


In [91]:
# CREAR EQUIPOS FRIO

def queryCreateTableEquiposFrio(conection_DB, nombre_tabla, dataframe):
    
    # ELIMINA BASE DE DATOS
    #conection_DB.connection.execute(f'DROP TABLE GestionProcesosTienda.dbo.{nombre_tabla};')
    #conection_DB.connection.commit()
    
    # CREA ARREGLO CON LAS COLUMNAS Y SUS TIPOS
    columns = []
    for index in dataframe.dtypes.index:
        tipo = str(dataframe.dtypes.loc[index])
        if tipo == "object":
            tipo = 'varchar'
        elif tipo == "int64":
            tipo = 'int'
        elif tipo == "float64":
            tipo = 'float'
        columns.append({'name': index, 'type': tipo})
    
    print(columns)
    
    # DA FORMATO A QUERY PARA CREAR TABLA 
    query = f"CREATE TABLE GestionProcesosTienda.dbo.{nombre_tabla} (\nid bigint IDENTITY(1,1) NOT NULL,"
    for column in columns:
        if column['type'] == 'varchar':
            query += f'\n{column["name"]} varchar(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL,'
        elif column['type'] == 'int':
            query += f'\n{column["name"]} bigint NOT NULL,'
    query += f"\ncreated_at datetime NOT NULL, \nupdated_at datetime NOT NULL, \nCONSTRAINT PK_{nombre_tabla}_3213E83F1717089E PRIMARY KEY (id)\n);"

    # SI LA TABLA NO EXISTE EN LA BASE DE DATOS LA CREA
    tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
    if 'equipos_frio_DS' not in tablas.TABLE_NAME.unique():
        # CREA TABLA
        conection_DB.connection.execute(query)
        conection_DB.connection.commit()

        # AGREGA LAS COLUMNAS DE FECHA DE CREACION Y ACTUALIZACION
        dataframe['created_at']   = datetime.today()
        dataframe['updated_at']   = datetime.today()

        # INSERTA DATOS EN BASE DE DATOS
        db_params               = urllib.parse.quote_plus(conection_DB.dataConnection)
        engine                  = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
        dataframe.to_sql('equipos_frio_DS', con=engine, index=False, if_exists="append", schema="dbo")

In [92]:
display(f'{rutaRelativa}/ROTACION/EQUIPOS.cvs')

'/Users/camilo/Desktop/PROYECTOS/ROTACION/EQUIPOS.cvs'

In [99]:
dataframe = pd.read_csv(f'{rutaRelativa}/ROTACION/EQUIPOS.csv')
dataframe = dataframe
#dataframe = dataframe.drop(['EAN','SECCION','ALMACENAMIENTO','Fecha Merma','SECCION','N° Doc. Merma','ZONA','Creado por','Modificado','Creado','NO_CONFORMIDAD','CANTIDAD','ESTADO_RETIRO','FECHA_RETIRO','DESTINO','MES','ID_SECCION','LIMPIEZA','FECHA_SOLICITUD','PROVEEDOR','UBICACION','FECHA_VENCIMIENTO' ], axis=1)
dataframe = dataframe.replace({np.nan: 'None'})
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: pd.Series(str(x).split(".")[0]))
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: '0' + x if(int(x) < 10) else x)

# MODIFICAR NOMBRES DE COLUMAS
#dataframe = dataframe.drop('ID', axis=1)
dataframe = dataframe.rename({
    'TIPO DE EQUIPO'    :'TIPO_DE_EQUIPO',
    'ID'                :'OTRO'
    },axis=1)

display(dataframe)
dataframe.dtypes

,nombreMaquina,temperaturaMinima,temperaturaMaxima,nombreLocal,TIPO_DE_EQUIPO,ZONA,NUMERO_SERIE,ESTADO,OTRO,OBSERVACION,MODELO,Creado,DESCRIPCION,ESTADO_CORREO,ORDEN_COMPRA,Modificado,Observaciones
0,1,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,1792484,OPERATIVO,405,None,mimet vv-70,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
1,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,1744947,OPERATIVO,406,None,mimet vv-70,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
2,3,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,1745070,OPERATIVO,407,None,mimet vv-70,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
3,4,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,2104168533,OPERATIVO,408,None,exkal ivn 2-4.3,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
4,5,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,2104168530,OPERATIVO,409,None,exkal ivn 2-4.3,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2,-18,-12,CHJ5088800 - VIÑA,Equipo Congelado,Congelados,None,PENDIENTE DE REVISIÓN,498,None,None,07/02/2023 13:28,None,None,None,07/02/2023 13:28,None
94,1,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Otros,1571204,OPERATIVO,499,cajas J7,mimet vv-8,20/02/2023 12:02,None,None,None,20/02/2023 12:03,None
95,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Otros,1809116,OPERATIVO,505,merma fiambrería,mimet vv-7a,20/02/2023 12:20,None,None,None,27/02/2023 10:30,None
96,3,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Otros,1834239,OPERATIVO,506,hielo cajas,mimet vv-38btf,20/02/2023 12:20,None,None,None,27/02/2023 10:30,None


nombreMaquina         int64
temperaturaMinima     int64
temperaturaMaxima     int64
nombreLocal          object
TIPO_DE_EQUIPO       object
ZONA                 object
NUMERO_SERIE         object
ESTADO               object
OTRO                  int64
OBSERVACION          object
MODELO               object
Creado               object
DESCRIPCION          object
ESTADO_CORREO        object
ORDEN_COMPRA         object
Modificado           object
Observaciones        object
dtype: object

In [94]:
display(dataframe)
dataframe.dtypes

,nombreMaquina,temperaturaMinima,temperaturaMaxima,nombreLocal,TIPO_DE_EQUIPO,ZONA,NUMERO_SERIE,ESTADO,OTRO,OBSERVACION,MODELO,Creado,DESCRIPCION,ESTADO_CORREO,ORDEN_COMPRA,Modificado,Observaciones
0,1,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,1792484,OPERATIVO,405,None,mimet vv-70,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
1,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,1744947,OPERATIVO,406,None,mimet vv-70,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
2,3,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,1745070,OPERATIVO,407,None,mimet vv-70,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
3,4,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,2104168533,OPERATIVO,408,None,exkal ivn 2-4.3,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
4,5,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Refrigerados,2104168530,OPERATIVO,409,None,exkal ivn 2-4.3,02/02/2023 6:13,None,None,None,02/02/2023 6:13,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2,-18,-12,CHJ5088800 - VIÑA,Equipo Congelado,Congelados,None,PENDIENTE DE REVISIÓN,498,None,None,07/02/2023 13:28,None,None,None,07/02/2023 13:28,None
94,1,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Otros,1571204,OPERATIVO,499,cajas J7,mimet vv-8,20/02/2023 12:02,None,None,None,20/02/2023 12:03,None
95,2,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Otros,1809116,OPERATIVO,505,merma fiambrería,mimet vv-7a,20/02/2023 12:20,None,None,None,27/02/2023 10:30,None
96,3,0,5,CHJ5028800 - DARKSTORE KENNEDY,Equipo Refrigerado,Otros,1834239,OPERATIVO,506,hielo cajas,mimet vv-38btf,20/02/2023 12:20,None,None,None,27/02/2023 10:30,None


nombreMaquina         int64
temperaturaMinima     int64
temperaturaMaxima     int64
nombreLocal          object
TIPO_DE_EQUIPO       object
ZONA                 object
NUMERO_SERIE         object
ESTADO               object
OTRO                  int64
OBSERVACION          object
MODELO               object
Creado               object
DESCRIPCION          object
ESTADO_CORREO        object
ORDEN_COMPRA         object
Modificado           object
Observaciones        object
dtype: object

In [95]:
#dataframe = pd.read_csv(f'{rutaRelativa}/ROTACION/TOTALES RETIRADOS.csv')
# MODIFICAR NOMBRES DE COLUMAS
#dataframe = dataframe.rename({
#    'CANTIDAD SKU RETIRADOS'    :'CANTIDAD_SKU_RETIRADOS',
#    'CANTIDAD SKU ESCANEADOS'   :'CANTIDAD_SKU_ESCANEADOS',
#    'PORCENTAJE RETIRADOS'      :'PORCENTAJE_RETIRADOS',
#    'PORCENTAJE TOTAL'          :'PORCENTAJE_TOTAL',
#    'Rubro'                     :'ALMACENAMIENTO',
#    'Creado por'                :'Creado_por',
#    },axis=1)

# ELIMINAR COLUMNAS
#dataframe = dataframe.drop(['Title', 'ZONA', 'ID'], axis=1)
#dataframe = dataframe.replace({np.nan: 'None'})

# LIMPIEZA DE ID_SECCION
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: pd.Series(str(x).split(".")[0]))
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: '0' if(x == 'None') else x)
#dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: '0' + x if(int(x) < 10) else x)

# LIMPIEZA CANTIDAD_SKU_RETIRADOS
#dataframe['CANTIDAD_SKU_RETIRADOS'] = dataframe["CANTIDAD_SKU_RETIRADOS"].apply(lambda x: pd.Series(str(x).split(".")[0]))
#dataframe['CANTIDAD_SKU_ESCANEADOS'] = dataframe["CANTIDAD_SKU_ESCANEADOS"].apply(lambda x: pd.Series(str(x).split(".")[0]))
#dataframe['Vencidos'] = dataframe["Vencidos"].apply(lambda x: pd.Series(str(x).split(".")[0]))

#dataframe = dataframe.head(10)
#display(dataframe)
#dataframe.dtypes

In [96]:
# EJECUTA FUNCION  EQUIPOS FRIO
nombre_tabla = 'equipos_frio_DS'
queryCreateTableEquiposFrio(dbAzure, nombre_tabla, dataframe)


[{'name': 'nombreMaquina', 'type': 'int'}, {'name': 'temperaturaMinima', 'type': 'int'}, {'name': 'temperaturaMaxima', 'type': 'int'}, {'name': 'nombreLocal', 'type': 'varchar'}, {'name': 'TIPO_DE_EQUIPO', 'type': 'varchar'}, {'name': 'ZONA', 'type': 'varchar'}, {'name': 'NUMERO_SERIE', 'type': 'varchar'}, {'name': 'ESTADO', 'type': 'varchar'}, {'name': 'OTRO', 'type': 'int'}, {'name': 'OBSERVACION', 'type': 'varchar'}, {'name': 'MODELO', 'type': 'varchar'}, {'name': 'Creado', 'type': 'varchar'}, {'name': 'DESCRIPCION', 'type': 'varchar'}, {'name': 'ESTADO_CORREO', 'type': 'varchar'}, {'name': 'ORDEN_COMPRA', 'type': 'varchar'}, {'name': 'Modificado', 'type': 'varchar'}, {'name': 'Observaciones', 'type': 'varchar'}]


/var/folders/rs/5wl8vzfj4vz7xxdjjr9hk3800000gn/T/ipykernel_40536/3650849646.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)


In [97]:
# EJECUTA FUNCION HISTORITO EQUIPOS FRIO DS
nombre_tabla = 'equipos_frio_DS'
queryCreateTableHistorico_equipos_DS(dbAzure, nombre_tabla, dataframe)

[{'name': 'nombreMaquina', 'type': 'int'}, {'name': 'temperaturaMinima', 'type': 'int'}, {'name': 'temperaturaMaxima', 'type': 'int'}, {'name': 'nombreLocal', 'type': 'varchar'}, {'name': 'TIPO_DE_EQUIPO', 'type': 'varchar'}, {'name': 'ZONA', 'type': 'varchar'}, {'name': 'NUMERO_SERIE', 'type': 'varchar'}, {'name': 'ESTADO', 'type': 'varchar'}, {'name': 'OTRO', 'type': 'int'}, {'name': 'OBSERVACION', 'type': 'varchar'}, {'name': 'MODELO', 'type': 'varchar'}, {'name': 'Creado', 'type': 'varchar'}, {'name': 'DESCRIPCION', 'type': 'varchar'}, {'name': 'ESTADO_CORREO', 'type': 'varchar'}, {'name': 'ORDEN_COMPRA', 'type': 'varchar'}, {'name': 'Modificado', 'type': 'varchar'}, {'name': 'Observaciones', 'type': 'varchar'}]


/var/folders/rs/5wl8vzfj4vz7xxdjjr9hk3800000gn/T/ipykernel_40536/931330936.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
